In [1]:
!pip install peft==0.14.0
!pip install -U datasets==3.3.2
!pip install wandb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 20.7 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 10.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.7 MB/s eta 0:00:00a 0:00:01
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 10.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
# 一个简单的推理函数
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"].to(model.device),
        attention_mask=inputs["attention_mask"].to(model.device),
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5, # 避免模型复读，默认值为1.0
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

In [3]:
import peft
from peft import LoraConfig, get_peft_model, PeftModel
import os
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer
import wandb
from datetime import datetime
import json
from datasets import load_dataset, load_from_disk

model_name = "bigscience/bloomz-560m"
# dataset = "noob123/imdb_review_3000"
dataset = "tatsu-lab/alpaca"

tokenizer = AutoTokenizer.from_pretrained(model_name)
#Create the Dataset to create prompts.
data = load_from_disk("./alpaca")
data = data.map(lambda samples: tokenizer(samples['output']), batched=True)
train_sample = data["train"]

train_sample = train_sample.remove_columns(['instruction', 'input', 'text'])

display(train_sample)

Dataset({
    features: ['output', 'input_ids', 'attention_mask'],
    num_rows: 52002
})

In [ ]:
# 加载checkpoint进行检查
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)
peft_model_path = os.path.join("./peft_outputs_r4_a4_d0.05_lr3e-05/checkpoint-2000")
loaded_model = PeftModel.from_pretrained(foundation_model, peft_model_path, is_trainable=False)
input_sentences = tokenizer("I love this movie because", return_tensors="pt")
foundational_outputs_sentence = get_outputs(loaded_model, input_sentences, max_new_tokens=100)
generated_text = tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True)[0]
print(generated_text)

I love this movie because it is so well made and the acting was excellent. I also loved that there were no sequels to follow, which makes me feel like I'm in a different world than most people who watch movies with such high expectations of what will happen next! This film definitely has some great moments but overall it's an amazing story about two teenagers trying out new things together.  It had lots more action scenes as they tried their hand at something else instead just being friends for now - that's cool!  The ending wasn't too


In [ ]:
# 完整的调参、训练、记录流程
log_file = "lora_experiments_alpaca.log"
with open(log_file, "w") as f:
    f.write("LoRA Experiment Log\n")
    f.write("="*50 + "\n")

def log_experiment(params, results):
    """记录实验参数和结果到日志文件"""
    with open(log_file, "a") as f:
        f.write(f"\nExperiment at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("-"*50 + "\n")
        f.write("PARAMETERS:\n")
        f.write(json.dumps(params, indent=4) + "\n")
        f.write("\nRESULTS:\n")
        f.write(results + "\n")
        f.write("="*50 + "\n")

# 定义要测试的参数组合
param_combinations = [
    {"r": 4, "lora_alpha": 4, "lora_dropout": 0.05, "learning_rate": 3e-5},
    {"r": 8, "lora_alpha": 8, "lora_dropout": 0.05, "learning_rate": 3e-5},
]

for params in param_combinations:
    # 初始化W&B (可选)
    wandb.init(
        project="lora_hyperparameter_tuning_alpaca",
        name=f"r{params['r']}_a{params['lora_alpha']}_d{params['lora_dropout']}_lr{params['learning_rate']}_qkv",
        config=params
    )
    
    try:
        # 1. 设置LoRA配置
        lora_config = LoraConfig(
            r=params["r"],
            lora_alpha=params["lora_alpha"],
            target_modules=["query_key_value"],
            lora_dropout=params["lora_dropout"],
            bias="lora_only",
            task_type="CAUSAL_LM"
        )
        
        # 2. 创建PEFT模型
        foundation_model = AutoModelForCausalLM.from_pretrained(model_name)
        peft_model = get_peft_model(foundation_model, lora_config)
        
        # 3. 设置输出目录
        working_dir = './'
        output_directory = os.path.join(working_dir, f"peft_outputs_r{params['r']}_a{params['lora_alpha']}_d{params['lora_dropout']}_lr{params['learning_rate']}")
        os.makedirs(output_directory, exist_ok=True)
        
        # 4. 训练参数
        training_args = TrainingArguments(
            output_dir=output_directory,
            report_to="wandb",
            run_name=f"r{params['r']}_a{params['lora_alpha']}_d{params['lora_dropout']}_lr{params['learning_rate']}",
            auto_find_batch_size=True,
            learning_rate=params["learning_rate"],
            num_train_epochs=2,
            use_cpu=False,
        )
        
        # 5. 训练器
        trainer = Trainer(
            model=peft_model,
            args=training_args,
            train_dataset=train_sample,
            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
        )
        
        # 6. 训练
        print(f"\nStarting training with params: {params}")
        trainer.train()
        
        # 7. 保存模型
        peft_model_path = os.path.join(output_directory, "lora_model")
        trainer.model.save_pretrained(peft_model_path)
        
        # 8. 加载模型并生成示例
        loaded_model = PeftModel.from_pretrained(foundation_model, peft_model_path, is_trainable=False)
        input_sentences = tokenizer("I love this movie because", return_tensors="pt")
        foundational_outputs_sentence = get_outputs(loaded_model, input_sentences, max_new_tokens=100)
        generated_text = tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True)[0]
        
        # 9. 记录结果
        results = f"Generated text:\n{generated_text}\n\nTraining metrics:\n{trainer.state.log_history[-1]}"
        log_experiment(params, results)
        
        print(f"Completed experiment with params: {params}")
        print(f"Generated text: {generated_text}")
        
    except Exception as e:
        error_msg = f"Failed experiment with params {params}. Error: {str(e)}"
        with open(log_file, "a") as f:
            f.write("\nERROR:\n")
            f.write(error_msg + "\n")
        print(error_msg)
    
    finally:
        # 确保W&B运行结束
        wandb.finish()

print("\nAll experiments completed. Results logged to:", log_file)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Starting training with params: {'r': 4, 'lora_alpha': 4, 'lora_dropout': 0.05, 'learning_rate': 3e-05}


Step,Training Loss


Step,Training Loss
500,2.749200
1000,2.669200


Step,Training Loss
500,2.741800
1000,2.741400
1500,2.741500
2000,2.690700
2500,2.710200
3000,2.663200
3500,2.731300
4000,2.741600
4500,2.676200
5000,2.684800


...

Step,Training Loss
500,2.710300
1000,2.652900


Step,Training Loss
500,2.733600
1000,2.728200
1500,2.735600
2000,2.679800
2500,2.698400
3000,2.650200
3500,2.726200
4000,2.744700
4500,2.693100
5000,2.705900


Completed experiment with params: {'r': 8, 'lora_alpha': 8, 'lora_dropout': 0.05, 'learning_rate': 3e-05}
Generated text: I love this movie because it is a great story. It tells the true and darker side of what happens when you are in need, with lots to do before your next step can be reached.
The film also has some touching moments that make me smile every time I watches its shots.  The characters have their own quirks as well which makes for an interesting experience watching them grow up. 
It was nice seeing how each character works out on different fronts so we could see who’s best friends or those people they’re dating


train/epoch,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇█████
train/grad_norm,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▂▁▁ ▁ ▂
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂▁▂▁▂▅▇█▇▆▅▆▅▅▅
total_flos,1.600198363373568e+16
train/epoch,2
train/global_step,52002
train/grad_norm,123590475776.0
train/learning_rate,0.0
train/loss,3.3365



All experiments completed. Results logged to: lora_experiments_alpaca.log
